In [54]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-02-22 22:11:39--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8001::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.4’

input.txt.4         100%[===================>]   1.06M  1.54MB/s    in 0.7s    

2023-02-22 22:11:40 (1.54 MB/s) - ‘input.txt.4’ saved [1115394/1115394]



In [55]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)

In [56]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [57]:
len(text)

1115394

In [58]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



All unique chars in text

In [59]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


Tokenize characters

In [60]:
text_int = {ch:i for i, ch in enumerate(chars)}
int_text = {i:ch for i, ch in enumerate(chars)}

encode = lambda x: [text_int[i] for i in x]
decode  = lambda x: ''.join([int_text[i] for i in x])

In [61]:
encode('hi there')

[46, 47, 1, 58, 46, 43, 56, 43]

In [62]:
decode(encode('hi there'))

'hi there'

In [63]:
data = torch.tensor(encode(text), dtype=torch.long)

Split Data

In [64]:
n = int(0.9*len(text))
train_data = data[:n]
valid_data = data[n:]

In [65]:
block_size = 8
data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [66]:
x = data[:block_size]
y = data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'context is {context} and target is {target}')

context is tensor([18]) and target is 47
context is tensor([18, 47]) and target is 56
context is tensor([18, 47, 56]) and target is 57
context is tensor([18, 47, 56, 57]) and target is 58
context is tensor([18, 47, 56, 57, 58]) and target is 1
context is tensor([18, 47, 56, 57, 58,  1]) and target is 15
context is tensor([18, 47, 56, 57, 58,  1, 15]) and target is 47
context is tensor([18, 47, 56, 57, 58,  1, 15, 47]) and target is 58


In [67]:
torch.manual_seed(43)
batch_size=4 #how many independent seq to process in parallel
block_size=8 #sets the size of the context

def get_batch(split):
    '''
        Generate a small batch of data of input x and target y
    '''

    data = train_data if split == 'train' else valid_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print(xb.shape)
print(yb.shape)



torch.Size([4, 8])
torch.Size([4, 8])


In [68]:
xb

tensor([[43,  1, 58, 53,  1, 51, 43,  6],
        [ 1, 57, 46, 43,  1, 61, 39, 57],
        [39, 42, 43,  1, 46, 47, 51,  1],
        [53, 59, 56,  1, 45, 56, 39, 41]])

In [69]:
yb

tensor([[ 1, 58, 53,  1, 51, 43,  6,  1],
        [57, 46, 43,  1, 61, 39, 57,  1],
        [42, 43,  1, 46, 47, 51,  1, 58],
        [59, 56,  1, 45, 56, 39, 41, 43]])

In [70]:
for b in range(batch_size): #block size
    for c in range(block_size):
        context = xb[b, :c+1]
        target = yb[b, c]
        print(f"Context:: {context}, Target:: {target}")



Context:: tensor([43]), Target:: 1
Context:: tensor([43,  1]), Target:: 58
Context:: tensor([43,  1, 58]), Target:: 53
Context:: tensor([43,  1, 58, 53]), Target:: 1
Context:: tensor([43,  1, 58, 53,  1]), Target:: 51
Context:: tensor([43,  1, 58, 53,  1, 51]), Target:: 43
Context:: tensor([43,  1, 58, 53,  1, 51, 43]), Target:: 6
Context:: tensor([43,  1, 58, 53,  1, 51, 43,  6]), Target:: 1
Context:: tensor([1]), Target:: 57
Context:: tensor([ 1, 57]), Target:: 46
Context:: tensor([ 1, 57, 46]), Target:: 43
Context:: tensor([ 1, 57, 46, 43]), Target:: 1
Context:: tensor([ 1, 57, 46, 43,  1]), Target:: 61
Context:: tensor([ 1, 57, 46, 43,  1, 61]), Target:: 39
Context:: tensor([ 1, 57, 46, 43,  1, 61, 39]), Target:: 57
Context:: tensor([ 1, 57, 46, 43,  1, 61, 39, 57]), Target:: 1
Context:: tensor([39]), Target:: 42
Context:: tensor([39, 42]), Target:: 43
Context:: tensor([39, 42, 43]), Target:: 1
Context:: tensor([39, 42, 43,  1]), Target:: 46
Context:: tensor([39, 42, 43,  1, 46]), 

BiGram Language Model

In [71]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()

        #each token directly reads the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        #idx and targets are both (Batch ,Time) tensor of integers
        logits = self.token_embedding_table(idx) # (Batch=batch_size, Time=block_size, Channel=vocab_size)

        if targets is not None:
            B, T, C = logits.shape

            #For pytorch loss module, reshaping
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        else:
            loss = None

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is B,T array in current context
        for _ in range(max_new_tokens):
            #get the preds
            logits, loss = self(idx)
            #get the last time step
            logits = logits[:, -1, :] # B,C
            #apply softmax to get probabilities
            probs = F.softmax(logits, dim=1) # B,C
            #sample from dist
            idx_next = torch.multinomial(probs, num_samples=1) #B,1
            #append
            idx = torch.cat((idx, idx_next), dim=1) #B,T+1
        return idx



m = BigramLanguageModel(vocab_size)
out, loss = m(xb,yb)
print(out.shape)
print(loss)

print(decode(m.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8305, grad_fn=<NllLossBackward0>)

MQks' n.ZFgZfBY;bt,q3uBeWTMT,Ji3.wPNI,i3wUA;LyutIrtbH&qdJbCRvqDg'TllFvmRv&qiuLBa:UEYBH'QqDIlfBGzzgY 


Train the Model

In [72]:
#Optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [73]:
batch_size=32
for epoch in range(1000):

    #sample a batch of data 
    xb, yb = get_batch('train')

    #eval loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

3.8411920070648193


In [74]:
print(decode(m.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=500)[0].tolist()))



O BRTy,CSo.mpO&RYhee
JwiXN;l ru.;WQerIw$S;PO:
was-$vrwR'd,QZ.UdxxFVjfasWcKjcME. $lut!DU p!k's,itF!itITZXeas-wRlvUrqG3&MO:&KwRDMrue:UF's-ce,MkTb:phDAUUzS vEN;wa'evxYeXebKpX-B&ey,X
A?lLhnwWEzVJno33omoawwRwn'e
U;KrZ3vj:&irgik'W:r,
MZFMzzgheloJqTO&LQObGuYn $v
C:wQKCaCEu.' PTHlO:JJitPRut?odGyeTYp!PqThjXd?W;fna?Fgl,J3A!JT-sese's-xNUFo,A,dxac?
e,.Dom mcZVqj!ZBAZg
AheY&R'raHosct $Q?Pm3yofXBdGMsww.,3z$vC.Sxt,OFaG,C$mLJiWx nbi3fwaJDTO UW; ePme,.!JBR !py OXgqDtc&CKCILoO lzUigw
Er,,Mb:p;!?sGclDoy,'QdU'shN'


### Self Attention Mathematical trick

In [75]:
B, T, C = 4, 8, 2 #Batch, Time, Channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

Provide info to the current token with all the past tokens: i.e average

In [76]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b,t] = torch.mean(xprev, 0)

In [77]:
x[0]

tensor([[-1.4102, -0.5717],
        [ 1.1007, -1.0828],
        [-0.5763,  0.7367],
        [ 1.0083, -0.0829],
        [-0.1793,  0.0455],
        [-0.7755, -0.0195],
        [ 1.7594, -2.3251],
        [ 0.6515,  0.2799]])

In [78]:
xbow[0]

tensor([[-1.4102, -0.5717],
        [-0.1547, -0.8272],
        [-0.2953, -0.3059],
        [ 0.0306, -0.2502],
        [-0.0113, -0.1910],
        [-0.1387, -0.1624],
        [ 0.1324, -0.4714],
        [ 0.1973, -0.3775]])

In [89]:
wts = torch.tril(torch.ones(T,T)) 
wts = wts / torch.sum(wts, 1, keepdim=True)
wts

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

Weighted sums with this trick

In [90]:
xbow2 = wts @ x # (T, T) @ (B, T, C) --> pytorch will apply batch mul
xbow2

tensor([[[-1.4102, -0.5717],
         [-0.1547, -0.8272],
         [-0.2953, -0.3059],
         [ 0.0306, -0.2502],
         [-0.0113, -0.1910],
         [-0.1387, -0.1624],
         [ 0.1324, -0.4714],
         [ 0.1973, -0.3775]],

        [[-0.6888,  0.2200],
         [ 0.4613,  0.4041],
         [ 0.4552,  0.1469],
         [ 0.5160,  0.1040],
         [ 0.1207,  0.1311],
         [ 0.3106,  0.3268],
         [ 0.1369,  0.2926],
         [ 0.1210,  0.1760]],

        [[ 0.2834,  0.0684],
         [ 0.8274,  0.9321],
         [ 0.1833,  0.2627],
         [ 0.2164,  0.3528],
         [-0.1003,  0.3226],
         [-0.0872, -0.0603],
         [-0.1843,  0.2428],
         [-0.2052,  0.0841]],

        [[-0.6858, -0.2193],
         [-0.8891, -0.2261],
         [-0.6727, -0.2258],
         [-0.7427, -0.3154],
         [-0.7512,  0.0753],
         [-0.6185, -0.1458],
         [-0.4669, -0.2838],
         [-0.2894, -0.1748]]])

Version with Softmax

In [93]:
tril = torch.tril(torch.ones(T, T))
wts = torch.zeros((T,T))
wts = wts.masked_fill(tril == 0, float('-inf'))
wts = F.softmax(wts, dim=-1)
wts

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [94]:
xbow3 = wts @ x
torch.allclose(xbow, xbow3)

True